In [1]:
# Third-Party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget
from IPython.display import IFrame
from wand.image import Image as WImage

# Local imports
from util_loads import Propeller, Airfoil, Loadcase

# IFrame("figures/latex/uml_lastrechnung.pdf", width=600, height=300)

----
<img src="figures/lwx.png" alt="Logo LWX" title="" style="height:100px" align="right"/>
<img src="figures/logo_IFL.svg" alt="Logo IFL" title="" style="height:100px" align="left"/>

## Entwicklung eines automatisierten Auslegungs- und Optimierungsprogramms für anisotrope UAV-Propeller

## Kolloquium zur Masterarbeit

### Lukas Hilbers, 26. April 2021
### E-Mail: l.hi@posteo.de

<div style="text-align: right"> 1 / XX </div>

---
## Agenda
- ### Motivation und Methodik
- ### Vorstellung der Anwendung
- ### Implementierung eines Lastrechnungstools
- ### FE-Modellierung in Python
- ### Automatisierte Strukturoptimierung

<div style="text-align: right"> 2 / XX </div>

---
## Motivation und Methodik
<img src="figures/Helics_Mat_1.jpg" alt="Naturfasern" title="" style="height:400px" align="right"/>

- ### Studienarbeit: Evaluierung naturfaserverstärkter Kunststoffe für Einsatz in Multicopter-Propellern
    * ### FE-Simulationen mit generischem Modell mit ANSYS-APDL
    * ### Werkstoffe (insbes. FLAXPREG) prinzipiell geeignet
    * ### Offene Themen: Lastrechnung und Übertragung auf reale Geometrien
    * ### Workflow: Weitgehend händisch, umständlich
- ### Interessengebiete: Optimierung und Programmierung
- ### Idee: Entwicklung einer automatisierten Toolchain zur Auslegung in Python
    * ### PyMAPDL: interaktives Interface zwischen Python und Ansys
    * ### PyOpt: wissenschaftlich fundierte Optimierungs-Toolbox

<div style="text-align: right"> 3 / XX </div>

---
## Vorstellung der Anwendung

<img src="figures/Propeller.png" alt="MF3218-Propeller" title="" style="height:200px" align="right"/>

- ### 4-Motoriger Multicopter
    * ### MTOW 25kg (Obergrenze für zulassungsfreie Systeme)
- ### Ausgewählte Propellergeometrie: T-Motor MF3218
    * ### Durchmesser $\approx 813\,mm$
    * ### Maximale Drehzahl $4000\,\frac{1}{min}$
    * ### Maximaler Schub $18\,kg$ / $177\,N$

<div style="text-align: right"> 4 / XX </div>

---

## Implementierung eines Lastrechnungstools

<img src="figures/xfoil_logo.gif" alt="Naturfasern" title="" style="height:100px" align="right"/>

- ### Anforderungen:
    * ### Verwendung beliebiger Grundrisse und Profile
    * ### Auswertung verschiedener Lastfälle und Bildung einhüllender Lastverläufe
    * ### Integration in Python-Workflow
    
- ### Gewählter Ansatz:
    * ### Nutzung (semi-)analytischer Verfahren wie Traglinientheorie nach Prandtl <br> bzw. Propellertheorie nach Betz
    * ### Guter Kompromiss aus Genauigkeit und numerischem Aufwand
    * ### Löser wie XFOIL und XROTOR frei verfügbar 

<div style="text-align: right"> 5 / XX </div>

---

## Implementierung eines Lastrechnungstools: Architektur

<img src="figures/latex/uml_lastrechnung.png" alt="Naturfasern" title="" style="height:300px" align="right"/>

- ### Objektorientierter Ansatz
- ### Airfoil-Klasse:
    * ### Repräsentiert aerodynamische Profile des Propellers
    * ### Berechnung von Polaren, Druckverteilungen, Straakprofilen
    * ### Anbindung an XFOIL als Löser
- ### Loadcase-Klasse:
    * ### Modellierung von Lastfällen
    * ### Vorgabe von Anströmgschwindigkeit, Drehzahl, Leistung, Schub...
- ### Propeller-Klasse:
    * ### Repräsentiert gesamten Propeller
    * ### Berechnung von (einhüllenden) Lastverläufen
    * ### Ausgabe lokaler Strömungsparameter, Profile, <br> Schnittkräfte und -momente

<div style="text-align: right"> 6 / XX </div>

---

## Implementierung eines Lastrechnungstools: Anwendung

In [2]:
# Define Airfoil
airfoil = Airfoil('mf3218.xfo', 500000, iter_limit=600)
airfoil.set_polar(alpha_start=-20, alpha_stop=20, alpha_inc=0.25)

# Instantiate Propeller and assign geometry and airfoils
propeller = Propeller(number_of_blades=2,
                      tip_radius=0.412,
                      hub_radius=0.04,
                      )

#                              [r/R, c/R, beta]
propeller.geometry = np.array([[0.10,0.078,0],
                               [0.121, 0.078, 0.],
                               [0.155, 0.100, 5.99],
                               [0.223, 0.160, 17.97],
                               [0.345, 0.149, 14.44],
                               [0.417, 0.142, 12.68],
                               [0.490, 0.135, 11.18],
                               [0.563, 0.128, 9.94],
                               [0.636, 0.121, 8.97],
                               [0.709, 0.114, 8.26],
                               [0.782, 0.107, 7.81],
                               [0.854, 0.100, 7.63],
                               [0.947, 0.091, 7.5],
                               [1., 0.066, 7.5],                               
                               ])

propeller.sections = [[0.121, airfoil],
                      [1., airfoil],]

# Instantiate Loadcases
loadcase = Loadcase(name='Max RPM', flight_speed=0.01)

# Assign Loadcase to Propeller Object
propeller.add_loadcase(loadcase)
propeller.loadcases[0][0].set_data('rpm',4000)

# Calculate loads
propeller.calc_loads()
propeller.set_load_envelope()

In [3]:
X, Y, Cp_suc, Cp_pres = propeller.pressure_distribution('envelope')

fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')

ax.plot_surface(X, Y, -Cp_suc + Cp_pres,cmap='viridis', edgecolor='none')

ax.set_xlabel('rel. Radius y/R / 1')
ax.set_ylabel('rel. Profiltiefe x/C / 1')
ax.set_zlabel('Druckbeiwert -C_p / 1')

/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt/util_loads/airfoil.py:297: RuntimeWarning: Unconverged point: Airfoil mf3218.xfo cl 0.873. Filling with 0.
  warnings.warn('Unconverged point: ' + str(self) + ' ' + \
/home/y0065120/Dokumente/Leichtwerk/Projects/helics-opt/util_loads/airfoil.py:297: RuntimeWarning: Unconverged point: Airfoil mf3218.xfo cl 0.873. Filling with 0.
  warnings.warn('Unconverged point: ' + str(self) + ' ' + \


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Druckbeiwert -C_p / 1')

---
## FE-Modellierung mit PyMapdl

<img src="figures/latex/uml_optimierung.png" alt="Naturfasern" title="" style="height:500px" align="right"/>

- ### PyMAPDL: Interaktives Python-ANSYS-Interface
- ### Anbindung an Lasten-Tool
- ### Optimierung mit Python-Bibliotheken